<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [117]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [118]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/remidion/PGM-Project
   2346ace..7779099  main       -> origin/main
Updating 2346ace..7779099
Fast-forward
 Timeliness.ipynb | 185 ++++++++++++++++++++++++++++++++++---------------------
 1 file changed, 116 insertions(+), 69 deletions(-)


# Load Data

In [121]:
out_df = pd.read_csv("PGM-Project/data/route_01_inbound_full.csv", index_col=[0,1], header=[0,1])
out_df

scheduled           ... timeliness         
stop_sequence                          1.0      2.0  ...        8.0      9.0
service_date route_start_time                        ...                    
2018-08-02   1900-01-01 05:10:00   18600.0  18780.0  ...       Late  On Time
             1900-01-01 05:30:00   19800.0  19980.0  ...       Late     Late
             1900-01-01 05:50:00   21000.0  21180.0  ...       Late     Late
             1900-01-01 06:06:00   21960.0  22140.0  ...       Late     Late
             1900-01-01 06:21:00   22860.0  23040.0  ...    On Time  On Time
...                                    ...      ...  ...        ...      ...
2019-09-30   1900-01-01 23:40:00   85200.0  85380.0  ...    On Time  On Time
             1900-01-01 23:54:00   86040.0  86220.0  ...    On Time  On Time
             1900-01-02 00:22:00   87720.0  87900.0  ...      Early    Early
             1900-01-02 00:37:00   88620.0  88800.0  ...    On Time    Early
             1900-01-02 00:52:00   89520.0  89700.0  ...    On Time    Early

[37203 rows x 18 columns]

In [136]:
out2_df = pd.read_csv("PGM-Project/data/route_01_inbound_binned_30min_6a1h.csv", index_col=[0], header=[0,1])
nb_bins_2 = int(out2_df["route_bin"].nunique())
out2_df

service_date scheduled  ... timeliness            route_bin
stop_sequence Unnamed: 1_level_1       1.0  ...        9.0  Unnamed: 20_level_1
0                     2018-08-03   21960.0  ...    On Time  1900-01-01 06:00:00
1                     2018-08-03   23700.0  ...       Late  1900-01-01 06:30:00
2                     2018-08-03   25560.0  ...       Late  1900-01-01 07:00:00
3                     2018-08-03   27300.0  ...      Early  1900-01-01 07:30:00
4                     2018-08-03   28980.0  ...       Late  1900-01-01 08:00:00
...                          ...       ...  ...        ...                  ...
9153                  2019-09-30   81240.0  ...    On Time  1900-01-01 22:30:00
9154                  2019-09-30   83580.0  ...      Early  1900-01-01 23:00:00
9155                  2019-09-30   85200.0  ...    On Time  1900-01-01 23:30:00
9156                  2019-09-30   87720.0  ...      Early  1900-01-02 00:00:00
9157                  2019-09-30   88620.0  ...      Early  1900-01-02 00:30:00

[9158 rows x 20 columns]

In [135]:
out3_df = pd.read_csv("PGM-Project/data/route_01_inbound_binned_20min_7a18h.csv", index_col=[0], header=[0,1])
nb_bins_3 = int(out3_df["route_bin"].nunique())
out4_df = pd.read_csv("PGM-Project/data/route_01_inbound_binned_30min_7a18h.csv", index_col=[0], header=[0,1])
nb_bins_4 = int(out4_df["route_bin"].nunique())
out5_df = pd.read_csv("PGM-Project/data/route_01_outbound_binned_30min_7a18h.csv", index_col=[0], header=[0,1])
nb_bins_5 = int(out5_df["route_bin"].nunique())
out6_df = pd.read_csv("PGM-Project/data/route_01_outbound_binned_30min_7a18h.csv", index_col=[0], header=[0,1]) #To change to binned 20 minutes
nb_bins_6 = int(out6_df["route_bin"].nunique())

# Training

In [125]:
X = out_df['scheduled'].to_numpy()
y = out_df['timeliness'].to_numpy()

X2 = out2_df['scheduled'].to_numpy()
y2 = out2_df['timeliness'].to_numpy()

X3 = out3_df['scheduled'].to_numpy()
y3 = out3_df['timeliness'].to_numpy()

X4 = out4_df['scheduled'].to_numpy()
y4 = out4_df['timeliness'].to_numpy()

X5 = out5_df['scheduled'].to_numpy()
y5 = out5_df['timeliness'].to_numpy()

X6 = out6_df['scheduled'].to_numpy()
y6 = out6_df['timeliness'].to_numpy()

## Aligned Bins

In [126]:
# Split dataset into training set and test set
test_fraction = 0.2
split_idx = int(X.shape[0] * test_fraction)
X_train, X_test, y_train, y_test = X[:-split_idx], X[-split_idx:], y[:-split_idx], y[-split_idx:]

## Mode Baseline

In [127]:
from collections import Counter

def baseline(df, verbose=False):
  c = Counter(df.flatten())
  counts = np.array(list(c.values()))
  if verbose: print('Values: ', dict(c.items()))
  ratio = counts/sum(counts)
  mode_performance = max(ratio)
  if verbose: print('Distribution: ', ratio)
  return mode_performance

def b2(df, verbose=True):
  c = Counter(df.flatten())
  myDict = dict(c.items())
  if verbose: print(myDict)
  s = sum(c.values())
  key = max(myDict, key=myDict.get)
  baseline = f'{myDict[key]/s} for "{key}"'
  #print(key, myDict[key]/s)
  return baseline

#print(' y:', b2(y))
#print()
#print('y2:', b2(y2))
#print()

print('y2:', b2(y2_train))

{'Early': 11368, 'On Time': 35415, 'Late': 19223}
y2: 0.5365421325334061 for "On Time"


In [128]:
print(X_train.shape,X2_train.shape, X2_train.reshape(-1,9).shape)
print(y_train.shape,y2_train.shape, y2_train.reshape(-1,9).shape)

(29763, 9) (193, 38, 9) (7334, 9)
(29763, 9) (193, 38, 9) (7334, 9)


# Inference

## 2-d Markov chain


In [ ]:
def get_dtw(knots, x, homogenous=False):
  if homogenous: return 0
  for k in range(len(knots) - 1):
        if x >= knots[k] and x < knots[k + 1]:
            return k

def parametrization_2d(state_dict, X_tr, y_tr, knots=None, h_vect=None):
  
  if h_vect is None: h_vect = [True, True, True]
  HP = h_vect[0]
  HQ = h_vect[1]
  HT = h_vect[2]


  N, T = X_tr.shape

  # Initialize the Transition Matrix.ces
  K = 1 if HT else len(knots)-1
  LP = 1 if HP else T-1
  LQ = 1 if HQ else T
  
  
  P = np.zeros((LP,K,3,3))
  Q = np.zeros((LQ,K,3,3))
  pi = np.zeros((K, 3))

  for i in range(N):
    for j in range(T):
      actual_state = state_dict[y_tr[i,j]]
      dtw = get_dtw(knots, X_tr[i,j], HT)

      # Update pi
      if j == 0:
        pi[dtw, actual_state] += 1

      # Update P
      if j < T - 1:
        next_state = state_dict[y_tr[i, j + 1]]
        l = 0 if HP else j
        P[l, dtw, actual_state, next_state] +=  1

      #Update Q
      if i < N-1:  
        next_state = state_dict[y_tr[i + 1, j]]
        l = 0 if HQ else j
        Q[l, dtw, actual_state, next_state] +=  1

    
  for k in range(K):

    if sum(pi[k,:]) == 0:
      pi[k,:] = [0,1,0]  # Default is "On Time"
      #pi[k,:] = [1/3, 1/3, 1/3]
    else:
      pi[k,:] = pi[k,:] / sum(pi[k,:])
  
    for j in range(LQ):
      for r in range(3):
        if Q[j,k,r,:].sum() == 0: Q[j,k,r,:] = pi[k,:] #[1/3, 1/3, 1/3]            
      Q[j,k,:,:] = (Q[j,k,:,:].T/Q[j,k,:,:].sum(axis = 1)).T
    
    for j in range(LP):
      for r in range(3):
        if P[j,k,r,:].sum() == 0: P[j,k,r,:] = pi[k,:] #[1/3, 1/3, 1/3]
      P[j,k,:,:] = (P[j,k,:,:].T/P[j,k,:,:].sum(axis = 1)).T

  return pi, P, Q

def pred_stop(previous_sequence, state_dict, pi, P, Q, time_stamp, knots = None, h_vect = None, alpha=1, current_sequence=None):
  #x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q

  if h_vect is None: h_vect = [True, True, True]
  HP = h_vect[0]
  HQ = h_vect[1]
  HT = h_vect[2]

  T = previous_sequence.shape[0]
  next_state = []

  for t in range(T):
    l = 0 if HQ else t
    dtw = get_dtw(knots, time_stamp[t], HT)
    vertical = Q[l, dtw, state_dict[previous_sequence[t]], :]
    
    if t==0:
      horizontal = pi[dtw, :] 
    else:
      if current_sequence is None:
        current_state = state_dict[next_state[t-1]]
      else:
        current_state = state_dict[current_sequence[t-1]]
      l = 0 if HP else t-1
      horizontal = P[l, dtw, current_state, :]

    idx = np.argmax(alpha * horizontal + (1-alpha) * vertical)
    next_state.append(list(state_dict.keys())[idx])

  return next_state


In [ ]:
a=3
(a in [1,2,3])

True

In [ ]:
def experiment(data, hyperparams, verbose=True):

  train_trips = data[0]  # Unordered list of bus TRIPS - May be empty
  train_timeliness = data[1]  # Associated state labels ^ - May be empty

  if verbose: print('last continuous-time timestamp:', np.max(train_trips)/3600)

  #----- Parameters ----- 
  if verbose: print('==== Hyperparameters ====')
  homogenous  = hyperparams[0]
  if verbose: print('Homogenous =',homogenous)
  ## homogenous  = 0 : 2D homogenous 
  ## homogenous  = 1 : time-only homogenous
  ## homogenous  = 2 : P stop-only homogenous
  ## homogenous  = 3 : Q stop-only homogenous
  ## homogenous  = 4 : P & Q stop-only homogenous
  ## homogenous  = 5 : 2D inhomogenous
  P_stop_homogenous = (homogenous in [0,2,4])  
  Q_stop_homogenous = (homogenous in [0,3,4])  
  time_homogenous = (homogenous in [0,1])
  h_vect = [P_stop_homogenous, Q_stop_homogenous, time_homogenous]  
  if verbose: print('P Stop Homogenous =',P_stop_homogenous)
  if verbose: print('Q Stop Homogenous =',Q_stop_homogenous)
  if verbose: print('Time Homogenous =',time_homogenous)

  ### DTWs time limits (for inhomogenous chains) ###
  #[0...3600*H] where H = 26  => There are 26 "workable" hours for a bus trip
  inhomogenous_method = hyperparams[1]
  if verbose: print('Inhomogenous Method =', inhomogenous_method)
  if inhomogenous_method==0:
    #Peaks period
    knots = [0, 21600, 32400, 50400, 64800, 93600]
  elif inhomogenous_method==1:
    # Hourly
    knots = [3600*i for i in range(0, 26 + 1)] 

  direction = hyperparams[2]
  ## direction = 0 : horizontal
  ## direction = 1 : vertical
  ## direction = 2 : 2D
  if verbose: print('Conditional Dependance Direction =', direction)

  ## 2D combination factor x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q
  ## where 
  ##   t is the stop ID
  ##   h is the hour of the day
  ##   P is the horizontal transition matrix (stop_id-wise)
  ##   Q is the vertical transition matrix (time-wise)
  if direction == 0:
    alpha = 1
  elif direction == 1:
    alpha = 0
  elif direction == 2:
    alpha = hyperparams[3]  # Any value between 0 and 1
    if alpha < 0 or alpha > 1:
      raise Exception('alpha must be between 0 and 1')
  if verbose: print('alpha =', alpha)

  training_method = hyperparams[4]
  if verbose: print('Training Method =', training_method)
  ## training_method = 0 : sliding window
  ## training_method = 1 : pre-train + sliding window      

  if training_method == 0:
    gamma = 1
    valid_split = 1
  else:
    gamma = hyperparams[5]  # damping coefficient
    valid_split = hyperparams[6]  # validation set split: 0.2
  if verbose: print('gamma = ', gamma)
  if verbose: print('validation split = ', valid_split)

  slidewindow_len = hyperparams[7]  # Nb of previous days considered: 60
  if verbose: print('Sliding Window Size (days) = ', slidewindow_len)
  #----------------------

  #------- Reshape  ------- 
  D, H, T = train_trips.shape  # (Day, hour-of-the-day, bus-stop-ID)
  train_trips = train_trips.reshape(-1,T)
  train_timeliness = train_timeliness.reshape(-1,T)

  #------- Split Dataset ------- 
  split_idx = int(train_trips.shape[0] * valid_split)
  x_tr, x_seq = train_trips[:-split_idx], train_trips[-split_idx:]
  y_tr, y_seq = train_timeliness[:-split_idx], train_timeliness[-split_idx:]

  N, T = x_seq.shape  # (Contiguous-trips, bus-stop-ID)

  if verbose:
    print('==== Dataset ====')
    print(f'X: Timestamp of scheduled bus stop, Y: Timeliness on that day at that bus stop')
    print(x_seq.shape, y_seq.shape)
    print('split index', train_trips.shape[0] - split_idx)
    print(x_tr.shape, x_seq.shape, y_tr.shape, y_seq.shape)

  #-- Check clean Data --
  if x_seq.shape[0] == 0: 
    #raise Exception('Empty validation set, increase "valid_split"')
    print('Empty validation set, increase "valid_split"')
  if slidewindow_len > x_seq.shape[0] - H: 
    slidewindow_len = x_seq.shape[0] - H
    print(f'Sliding Window reduced to: {slidewindow_len/H} days')
  #----------------------


  #Init
  state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
  accu1 = 0
  accu2 = 0

  if training_method == 1:  # Batch mode: use training set to establish priors
    # Train
    pi_batch, P_batch, Q_batch = parametrization_2d(state_dict, x_tr, y_tr, knots, h_vect)


  if training_method == 0 or training_method == 1:  # Sliding window: use sequential data from test set to modify the baseline
    for i in range(slidewindow_len, N):
      # Create a validation set   
      X_seq = x_seq[i-slidewindow_len:i,:]
      Y_seq = y_seq[i-slidewindow_len:i,:]
      X_tst = x_seq[i,:]
      Y_tst = y_seq[i,:]
      # Train
      pi, P, Q = parametrization_2d(state_dict, X_seq, Y_seq, knots, h_vect)
      if training_method == 1:
        pi = gamma * pi + (1-gamma) * pi_batch
        P = gamma * P + (1-gamma) * P_batch
        Q = gamma * Q + (1-gamma) * Q_batch
      

      #Predict the complete sequence of states for the following trip given a trip
      future_states = pred_stop(Y_seq[-1], state_dict, pi, P, Q, X_tst, knots, h_vect, alpha)
      for t in range(T):
        if Y_tst[t] == future_states[t]:
            accu1 += 1

      #Predict the state of the next stop given the previous trip and the state of the current stop
      future_states = pred_stop(Y_seq[-1], state_dict, pi, P, Q, X_tst, knots, h_vect, alpha, Y_tst)
      for t in range(T):
          if Y_tst[t] == future_states[t]:
              accu2 += 1

      if i%100==0: print(f'accu1: {accu1}, accu2: {accu2}', end='\r')


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  if verbose:
    print('==== Results ====')
    print('Next Sequence:', accu1)
    print('Next State', accu2)
    print()
    print()
  return accu1, accu2

In [ ]:
from datetime import datetime as dt
# Set Hyperparemeters
homogenous = 0           # 0:2D homogenous 
                          # 1:time-only homogenous
                          # 2:P stop-only homogenous
                          # 3:Q stop-only homogenous
                          # 4:P & Q stop-only homogenous
                          # 5:2D inhomogenous

inhomogenous_method = 0  # 0:Peak periods, 1:Hourly
direction = 0             # 0:Horizontal, 1:Vertical, 2:2D
alpha = 0.5               # [0,0.5]:vertical (time) axis dominates, [0.5,1]:horizontal (stop_id) axis dominates
training_method = 0       # 0:sliding window only, 1: training set + sequ
gamma = 0.9               # [0,0.5]:static set of routes dominate, [0.5,1]:set of recent routes dominates
valid_split = 0.2         # (0,1]: subset of training set on which predictions are made (empty validation set is not allowed)
slidewindow_len = 60      # Nb of days from which the priors and transition matrix are computed

hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
data = [X2_train, y2_train]

print('hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]')
print('hyperparams,              Sequence Prediction, State Prediction')

fname = dt.now().strftime("%y%m%d_%H%M%S")
with open(f'PGM-Project/data/results_{fname}.txt', "w") as myfile:
  myfile.write('hyperparams, Sequence Prediction, State Prediction\n')

idx = 0

for homogenous in range(6):
  for inhomogenous_method in [0,1]:
    for direction in [0,1,2]:
      if direction == 2: ALPHA = [.1,.3,.5,.7,.9]
      else: ALPHA = [direction]
      for alpha in ALPHA:
        for training_method in [0,1]:
          if training_method == 0:
            GAMMA = [1]
            VSPLIT = [1]
          else:
            GAMMA = [.5, .7, .9]
            VSPLIT = [0.2, 0.5, 0.8]
          for gamma in GAMMA:
            for valid_split in VSPLIT:
              for slidewindow_len in [5, 10, 30, 60]:
                hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
                accu1, accu2 = experiment(data, hyperparams, verbose=False)
                idx += 1
                print(idx, hyperparams, accu1, accu2)
                with open(f'PGM-Project/data/results_{fname}.txt', "a") as myfile:
                  myfile.write(f"{idx}, {hyperparams}, {accu1}, {accu2}\n")


hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
hyperparams,              Sequence Prediction, State Prediction
1 [0, 0, 0, 0, 0, 1, 1, 5] 0.5361349888570519 0.7766559027304013
2 [0, 0, 0, 0, 0, 1, 1, 10] 0.5389131622064446 0.8042205230899934
3 [0, 0, 0, 0, 0, 1, 1, 30] 0.536798709991481 0.8195357186321042
4 [0, 0, 0, 0, 0, 1, 1, 60] 0.5380197354351878 0.8205633458589191
5 [0, 0, 0, 0, 1, 0.5, 0.2, 5] 0.549471442695262 0.7846224047456081
6 [0, 0, 0, 0, 1, 0.5, 0.2, 10] 0.5464743589743589 0.8089133089133089
7 [0, 0, 0, 0, 1, 0.5, 0.2, 30] 0.5430981120396162 0.8149953574744662
8 [0, 0, 0, 0, 1, 0.5, 0.2, 60] 0.5425162004109373 0.8151572625256835
9 [0, 0, 0, 0, 1, 0.5, 0.5, 5] 0.5549790642636082 0.7843315735178106
10 [0, 0, 0, 0, 1, 0.5, 0.5, 10] 0.553793333940996 0.8032996080576064
11 [0, 0, 0, 0, 1, 0.5, 0.5, 30] 0.5528060367213515 0.8094277945803928
12 [0, 0, 0, 0, 1, 0.5, 0.5, 60] 0.5527523642300465 0.809845054369

KeyboardInterrupt: ignored

##Tentative code

In [138]:
def prep_df(dataset):

  # Reshape into (Days x time bins x stops)
  X = dataset[1]
  y = dataset[2]
  n_bins = dataset[3]
  shape = (X.shape[0]//n_bins, n_bins, X.shape[1])
  X = X.reshape(shape) 
  y = y.reshape(shape) 

  # Split dataset #2 into training set and test set
  test_fraction = 0.2
  split_idx = int(X.shape[0] * test_fraction)
  X_train, X_test, y_train, y_test = X[:-split_idx], X[-split_idx:], y[:-split_idx], y[-split_idx:] 

  return [X, y]

In [130]:
def get_dtw(knots, x, homogenous=False):
  if homogenous: return 0
  for k in range(len(knots) - 1):
        if x >= knots[k] and x < knots[k + 1]:
            return k

def parametrization_2d(state_dict, X_tr, y_tr, knots=None, h_vect=None):

  if h_vect is None: h_vect = [True, True, True]
  HP = h_vect[0]
  HQ = h_vect[1]
  HTP = h_vect[2]
  HTQ = h_vect[3]


  D, S, T = X_tr.shape

  # Initialize the Transition Matrix.ces
  KP = 1 if HTP else len(knots)-1
  KQ = 1 if HTQ else len(knots)-1
  LP = 1 if HP else T-1
  LQ = 1 if HQ else T
  
  
  P = np.zeros((LP,KP,3,3))
  Q = np.zeros((LQ,KQ,3,3))
  pi_h = np.zeros((KP, 3))
  pi_v = np.zeros((LQ, 3))

  #update
  for i in range(D):
    for s in range(S):
      for j in range(T):
        actual_state = state_dict[y_tr[i,s,j]]
        dtwP = get_dtw(knots, X_tr[i,s,j], HTP)
        dtwQ = get_dtw(knots, X_tr[i,s,j], HTQ)

        # Update pi s
        if j == 0:
          pi_h[dtwP, actual_state] += 1
        
        if s == 0:
          l = 0 if HQ else j
          pi_v[l, actual_state] += 1

        # Update P
        if j < T - 1:
          next_state = state_dict[y_tr[i, s,  j + 1]]
          l = 0 if HP else j
          P[l, dtwP, actual_state, next_state] +=  1

        #Update Q
        if s < S-1:  
          next_state = state_dict[y_tr[i, s + 1, j]]
          l = 0 if HQ else j
          Q[l, dtwQ, actual_state, next_state] +=  1

  #normalize
  for l in range(LQ):  
    if sum(pi_v[l,:]) == 0:
      pi_v[l,:] = [0,1,0]  # Default is "On Time"
      #pi[k,:] = [1/3, 1/3, 1/3]
    else:
      pi_v[l,:] = pi_v[l,:] / sum(pi_v[l,:])
    
    for k in range(KQ):
      for r in range(3):
        if Q[l,k,r,:].sum() == 0: Q[l,k,r,:] = pi_v[l,:] #[1/3, 1/3, 1/3]            
      Q[l,k,:,:] = (Q[l,k,:,:].T/Q[l,k,:,:].sum(axis = 1)).T
  
  for k in range(KP):
    if sum(pi_h[k,:]) == 0:
      pi_h[k,:] = [0,1,0]  # Default is "On Time"
      #pi[k,:] = [1/3, 1/3, 1/3]
    else:
      pi_h[k,:] = pi_h[k,:] / sum(pi_h[k,:])

    for j in range(LP):
      for r in range(3):
        if P[j,k,r,:].sum() == 0: P[j,k,r,:] = pi_h[k,:] #[1/3, 1/3, 1/3]
      P[j,k,:,:] = (P[j,k,:,:].T/P[j,k,:,:].sum(axis = 1)).T

  return pi_h, pi_v, P, Q

def pred_stop(state_dict, pi_h, pi_v, s, P, Q, time_stamp, knots = None, h_vect = None, alpha=1, previous_sequence = None, current_sequence=None):
  #x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q
  if h_vect is None: h_vect = [True, True, True]
  HP = h_vect[0]
  HQ = h_vect[1]
  HTP = h_vect[2]
  HTQ = h_vect[3]

  T = time_stamp.shape[0]
  next_state = []

  for t in range(T):
    dtwP = get_dtw(knots, time_stamp[t], HTP)
    dtwQ = get_dtw(knots, time_stamp[t], HTQ)

    #horizontal
    if t==0:
      horizontal = pi_h[dtwP, :]
    else:
      if current_sequence is None:
        current_state = state_dict[next_state[t-1]]
      else:
        current_state = state_dict[current_sequence[t-1]]
      l = 0 if HP else t-1
      horizontal = P[l, dtwP, current_state, :]
    
    #vertical
    l = 0 if HQ else t
    if s == 0: 
      vertical = pi_v[l, :] 
    else:
      vertical = Q[l, dtwQ, state_dict[previous_sequence[t]], :]

    idx = np.argmax(alpha * horizontal + (1-alpha) * vertical)
    next_state.append(list(state_dict.keys())[idx])

  return next_state

In [131]:
def experiment(data, hyperparams, verbose=True):

  train_trips = data[0]  # Unordered list of bus TRIPS - May be empty
  train_timeliness = data[1]  # Associated state labels ^ - May be empty

  if verbose: print('last continuous-time timestamp:', np.max(train_trips)/3600)

  #----- Parameters ----- 
  if verbose: print('==== Hyperparameters ====')
  homogenous  = hyperparams[0]
  if verbose: print('Homogenous =',homogenous)
  ## homogenous  = 0 : all homogenous
  ## homogenous  = 1 : all homogeneous except Q stop
  ## homogenous  = 2 : all homogeneous except P stop
  ## homogenous  = 3 : all homogeneous except P & Q stop
  ## homogenous  = 4 : all homogeneous except time Q
  ## homogenous  = 5 : all homogeneous except time Q and stop Q
  ## homogenous  = 6 : all homogeneous except time Q and stop P
  ## homogenous  = 7 : all inhomogeneous except time P
  ## homogenous  = 8 : all homogeneous except time P
  ## homogenous  = 9 : all homogeneous except time P and stop Q
  ## homogenous  = 10 : all homogeneous except time P and stop P
  ## homogenous  = 11 : all inhomogeneous except time Q
  ## homogenous  = 12 : all homogeneous except time P & Q
  ## homogenous  = 13 : all inhomogeneous except stop P
  ## homogenous  = 14 : all inhomogeneous except stop Q
  ## homogenous  = 15 : all inhomogeneous

  P_time_homogenous = (homogenous in [0,1,2,3,4,5,6,7])  
  Q_time_homogenous = (homogenous in [0,1,2,3,8,9,10,11])  
  P_stop_homogenous = (homogenous in [0,1,4,5,8,9,12,13])
  Q_stop_homogenous = (homogenous in [0,2,4,6,8,10,12,14])

  h_vect = [P_stop_homogenous, Q_stop_homogenous, P_time_homogenous, Q_time_homogenous]  
  if verbose: print('P Stop Homogenous =',P_stop_homogenous)
  if verbose: print('Q Stop Homogenous =',Q_stop_homogenous)
  if verbose: print('P Time Homogenous =',P_time_homogenous)
  if verbose: print('Q Time Homogenous =',Q_time_homogenous)

  ### DTWs time limits (for inhomogenous chains) ###
  #[0...3600*H] where H = 26  => There are 26 "workable" hours for a bus trip
  inhomogenous_method = hyperparams[1]
  if verbose: print('Inhomogenous Method =', inhomogenous_method)
  if inhomogenous_method==0:
    #Peaks period
    knots = [0, 21600, 32400, 50400, 64800, 93600]
  elif inhomogenous_method==1:
    # Hourly
    knots = [3600*i for i in range(0, 26 + 1)] 

  direction = hyperparams[2]
  ## direction = 0 : horizontal
  ## direction = 1 : vertical
  ## direction = 2 : 2D
  if verbose: print('Conditional Dependance Direction =', direction)

  ## 2D combination factor x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q
  ## where 
  ##   t is the stop ID
  ##   h is the hour of the day
  ##   P is the horizontal transition matrix (stop_id-wise)
  ##   Q is the vertical transition matrix (time-wise)
  if direction == 0:
    alpha = 1
  elif direction == 1:
    alpha = 0
  elif direction == 2:
    alpha = hyperparams[3]  # Any value between 0 and 1
    if alpha < 0 or alpha > 1:
      raise Exception('alpha must be between 0 and 1')
  if verbose: print('alpha =', alpha)

  training_method = hyperparams[4]
  if verbose: print('Training Method =', training_method)
  ## training_method = 0 : sliding window
  ## training_method = 1 : pre-train + sliding window      

  if training_method == 0:
    gamma = 1
    valid_split = 1
  else:
    gamma = hyperparams[5]  # damping coefficient
    valid_split = hyperparams[6]  # validation set split: 0.2
  if verbose: print('gamma = ', gamma)
  if verbose: print('validation split = ', valid_split)

  slidewindow_len = hyperparams[7]  # Nb of previous days considered: 60
  if verbose: print('Sliding Window Size (days) = ', slidewindow_len)
  #----------------------

  #------- Reshape  ------- 
  D, H, T = train_trips.shape  # (Day, hour-of-the-day, bus-stop-ID)
  train_trips = train_trips
  train_timeliness = train_timeliness

  #------- Split Dataset ------- 
  split_idx = int(train_trips.shape[0] * valid_split)
  x_tr, x_seq = train_trips[:-split_idx], train_trips[-split_idx:]
  y_tr, y_seq = train_timeliness[:-split_idx], train_timeliness[-split_idx:]

  D, S, T = x_seq.shape  # (Days, time step, bus-stop-ID)

  if verbose:
    print('==== Dataset ====')
    print(f'X: Timestamp of scheduled bus stop, Y: Timeliness on that day at that bus stop')
    print(x_seq.shape, y_seq.shape)
    print('split index', train_trips.shape[0] - split_idx)
    print(x_tr.shape, x_seq.shape, y_tr.shape, y_seq.shape)
    print(x_seq.shape[0])

  #-- Check clean Data --
  if x_seq.shape[0] == 0: 
    #raise Exception('Empty validation set, increase "valid_split"')
    print('Empty validation set, increase "valid_split"')
 
  if slidewindow_len > x_seq.shape[0] : 
    slidewindow_len = x_seq.shape[0] - 10
    print(f'Sliding Window reduced to: {slidewindow_len} days')
  #----------------------


  #Init
  state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
  accu1 = 0
  accu2 = 0

  if training_method == 1:  # Batch mode: use training set to establish priors
    # Train
    pi_h_batch, pi_v_batch, P_batch, Q_batch = parametrization_2d(state_dict, x_tr, y_tr, knots, h_vect)


  if training_method == 0 or training_method == 1:  # Sliding window: use sequential data from test set to modify the baseline
    for i in range(slidewindow_len, D):
      # Create a validation set   
      X_seq = x_seq[i-slidewindow_len:i,:,:]
      Y_seq = y_seq[i-slidewindow_len:i,:,:]

      # Train
      pi_h, pi_v, P, Q = parametrization_2d(state_dict, X_seq, Y_seq, knots, h_vect)
      if training_method == 1:
        pi_h = gamma * pi_h + (1-gamma) * pi_h_batch
        pi_v = gamma * pi_v + (1-gamma) * pi_v_batch
        P = gamma * P + (1-gamma) * P_batch
        Q = gamma * Q + (1-gamma) * Q_batch
      
      for s in range(S):
        
        X_tst = x_seq[i,s,:]
        Y_tst = y_seq[i,s,:]
    
        #Predict the complete sequence of states for the following trip given a trip
        previous_sequence =y_seq[i,s - 1,:] if s > 0 else None
        future_states = pred_stop(state_dict, pi_h, pi_v, s, P, Q, X_tst, knots, h_vect, alpha, previous_sequence)
        for t in range(T):
          if Y_tst[t] == future_states[t]:
            accu1 += 1

        #Predict the state of the next stop given the previous trip and the state of the current stop
        future_states = pred_stop(state_dict, pi_h, pi_v, s, P, Q, X_tst, knots, h_vect, alpha, previous_sequence, Y_tst)
        for t in range(T):
            if Y_tst[t] == future_states[t]:
              accu2 += 1

        if i%100==0: print(f'accu1: {accu1}, accu2: {accu2}', end='\r')


  accu1 = accu1 /((D  - slidewindow_len) * S * T)
  accu2 = accu2 /((D - slidewindow_len) * S * T)
  if verbose:
    print('==== Results ====')
    print('Next Sequence:', accu1)
    print('Next State', accu2)
    print()
    print()
  return accu1, accu2

In [140]:
from datetime import datetime as dt
# Set Hyperparemeters
homogenous = 0           # 0:2D homogenous 
                          # 1:time-only homogenous
                          # 2:P stop-only homogenous
                          # 3:Q stop-only homogenous
                          # 4:P & Q stop-only homogenous
                          # 5:2D inhomogenous

inhomogenous_method = 0  # 0:Peak periods, 1:Hourly
direction = 0             # 0:Horizontal, 1:Vertical, 2:2D
alpha = 0.5               # [0,0.5]:vertical (time) axis dominates, [0.5,1]:horizontal (stop_id) axis dominates
training_method = 0       # 0:sliding window only, 1: training set + sequ
gamma = 0.9               # [0,0.5]:static set of routes dominate, [0.5,1]:set of recent routes dominates
valid_split = 0.2         # (0,1]: subset of training set on which predictions are made (empty validation set is not allowed)
slidewindow_len = 60     # Nb of days from which the priors and transition matrix are computed

hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
data = [X2_train, y2_train]

print('hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]')
print('hyperparams,              Sequence Prediction, State Prediction')

fname = dt.now().strftime("%y%m%d_%H%M%S")
with open(f'PGM-Project/data/results_{fname}.txt', "w") as myfile:
  myfile.write('hyperparams, Sequence Prediction, State Prediction\n')

idx = 0

for dataset in [["route_01_inbound_binned_20min_7a18h", X3, y3, nb_bins_3], ["route_01_inbound_binned_30min_7a18h.csv", X4, y4, nb_bins_4],
                ["route_01_outbound_binned_30min_7a18h.csv", X5, y5, nb_bins_5], ["route_01_outbound_binned_20min_7a18h.csv", X6, y6, nb_bins_6]]:
  data = prep_df(dataset)              
  for homogenous in range(16):
    for inhomogenous_method in [0,1]:
      for direction in [0,1,2]:
        if direction == 2: ALPHA = [.1,.3,.5,.7,.9]
        else: ALPHA = [direction]
        for alpha in ALPHA:
          for training_method in [0,1]:
            if training_method == 0:
              GAMMA = [1]
              VSPLIT = [1]
            else:
              GAMMA = [.5, .7, .9]
              VSPLIT = [0.2, 0.5, 0.8]
            for gamma in GAMMA:
              for valid_split in VSPLIT:
                for slidewindow_len in [5, 10, 30, 60]:
                  hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
                  accu1, accu2 = experiment(data, hyperparams, verbose=False)
                  idx += 1
                  print(dataset[0], idx, hyperparams, accu1, accu2)
                  with open(f'PGM-Project/data/results_{fname}.txt', "a") as myfile:
                    myfile.write(f"{dataset[0]}, {idx}, {hyperparams}, {accu1}, {accu2}\n")

hyperparams = [homogenous, inhomogenous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
hyperparams,              Sequence Prediction, State Prediction
route_01_inbound_binned_20min_7a18h 1 [0, 0, 0, 0, 0, 1, 1, 5] 0.51010101010101 0.7995303916356548
route_01_inbound_binned_20min_7a18h 2 [0, 0, 0, 0, 0, 1, 1, 10] 0.5116241782908449 0.7985066080304175
route_01_inbound_binned_20min_7a18h 3 [0, 0, 0, 0, 0, 1, 1, 30] 0.5163180360030754 0.795673268114213
route_01_inbound_binned_20min_7a18h 4 [0, 0, 0, 0, 0, 1, 1, 60] 0.517060640771981 0.7938838557395258
route_01_inbound_binned_20min_7a18h 5 [0, 0, 0, 0, 1, 0.5, 0.2, 5] 0.5426055426055426 0.7991452991452992
route_01_inbound_binned_20min_7a18h 6 [0, 0, 0, 0, 1, 0.5, 0.2, 10] 0.5510662177328844 0.8015071348404682
route_01_inbound_binned_20min_7a18h 7 [0, 0, 0, 0, 1, 0.5, 0.2, 30] 0.6094276094276094 0.8249158249158249
Sliding Window reduced to: 21 days
route_01_inbound_binned_20min_7a18h 8 [0, 0, 0, 0, 1, 0.5, 0.

KeyboardInterrupt: ignored

## 1-d Markov Chain



In [ ]:
import math

def get_dtw(knots, x, homogenous=False):
  if homogenous: return 0
  for k in range(len(knots) - 1):
        if x >= knots[k] and x < knots[k + 1]:
            return k


def parametrization(state_dict, X_tr, y_tr, knots = None, homogenous  = True):
      
    N, T = X_tr.shape

    # Initialize the Transition Matrix.ces
    K = 1 if homogenous  else len(knots)-1
    transition_matrix = np.zeros((K,3,3))
    pi = np.zeros((K, 3))

    for i in range(N):

      # Ground Truth for first stop of sequence
      actual_state = state_dict[y_tr[i,0]] 

      dtw = get_dtw(knots, X_tr[i,0], homogenous )
      pi[dtw, actual_state] += 1

      for j in range(T - 1):
        actual_state = state_dict[y_tr[i,j]]
        next_state = state_dict[y_tr[i,j + 1]]
        
        dtw = get_dtw(knots, X_tr[i,j], homogenous )
        transition_matrix[dtw, actual_state, next_state] +=  1

    for k in range(K):

      if sum(pi[k,:]) == 0:
        pi[k,:] = [0,1,0]  # Baseline is On Time
        #pi[k,:] = [1/3, 1/3, 1/3]
      else:
        pi[k,:] = pi[k,:] / sum(pi[k,:])
      
      #Check first divison by 0
      #if transition_matrix[dtw,:,:].sum(axis = 1)[0] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[1] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[2] == 0:
      #    transition_matrix[dtw,:,:] =  transition_matrix[dtw - 1,:,:]
      #else:
      if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
        transition_matrix[k,0,:] = pi[k,:] #[1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
        transition_matrix[k,1,:] = pi[k,:] #[1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
        transition_matrix[k,2,:] = pi[k,:] #[1/3, 1/3, 1/3]
          
      transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    return pi, transition_matrix


def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots = None, homogenous  = True):

    future_states = []

    dtw = get_dtw(knots, X_tst[0], homogenous )
    first_stop_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :])

    future_states.append(first_stop_state)

    for i in range(len(X_tst)):
      current_state = state_dict[future_states[-1]]

      dtw = get_dtw(knots, X_tst[i], homogenous )
      next_state = np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, current_state, :])
      future_states.append(next_state)

    return future_states


def pred_next_stop_delay(current_state, state_dict, transition_matrix, X_tst, knots, homogenous  = True):

    dtw = get_dtw(knots, X_tst, homogenous )
    return np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, state_dict[current_state], :])



In [ ]:
#----- Parameters ----- 
slidewindow_len = 60 #Number of previous days to train on
homogenous = True
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogenous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X_train.shape
for h in [True, False]:
  homogenous =h
  print()
  print('homogenous =',h)

  for k in range(slidewindow_len, N):

      X_tr = X_train[k-slidewindow_len:k,:]
      y_tr = y_train[k-slidewindow_len:k,:]
      X_tst = X_train[k,:]
      y_tst = y_train[k,:]
      pi, transition_matrix = parametrization(state_dict, X_tr, y_tr, knots, homogenous )


      #Predict the next trip sequence of states
      future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, homogenous )

      for t in range(T):
        if y_tst[t] == future_states[t]:
            accu1 += 1
      

      #Predict the state of the next stop
      dtw = get_dtw(knots, X_tst[0], homogenous )
      first_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :]) #Prediction of the first stop state is random
      
      if y_tst[0] == first_state:
          accu2 += 1

      for t in range(1, T):
          next_state = pred_next_stop_delay(y_tst[t-1], state_dict, transition_matrix, X_tst[t], knots, homogenous )
          
          if y_tst[t] == next_state:
              accu2 += 1


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  print('Sequence:', accu1)
  print('Next', accu2)



homogenous = True


KeyboardInterrupt: ignored

accu1 = Estimate whole sequence \\
accu2 = Estimate state for next stop ID \\
 \\
homogenous:

*   accu1: 0.43205470637966764
*   accu2: 0.6853260910226772

inhomogenous (DTW = 1 hour):

*   accu1 0.4616673774306257
*   accu2 0.6461181065248132

inhomogenous (DTW = peak vs off-peak):

*   accu1 0.44364015891185776
*   accu2 0.6807622382331157



##Old algo: resolve conflict by merging with above

In [ ]:
import math

def parametrization(state_dict, X_tr, y_tr, knots, time_homogenous = True, stop_homogenous = True):
      
    N, T = X_tr.shape

    if not time_homogenous:
        transition_matrix = np.zeros((len(knots) - 1,3,3))
        pi = np.zeros((len(knots) - 1, 3))

    elif not stop_homogenous:
        transition_matrix = np.zeros((T - 1,3,3))
        pi = np.zeros(3)
    
    else:
        transition_matrix = np.zeros((3,3))
        pi = np.zeros(3)

    for i in range(N):

        actual_state = state_dict[y_tr[i,0]] 

        if not time_homogenous:
            for k in range(len(knots) - 1):
                if (X_tr[i,0] >= knots[k]) and (X_tr[i,0] < knots[k + 1]):
                    dtw = k
            pi[dtw, actual_state] += 1
  
        else:
            pi[actual_state] += 1

        for j in range(T - 1):
            actual_state = state_dict[y_tr[i,j]]
            next_state = state_dict[y_tr[i,j + 1]]

            if not time_homogenous:
                for k in range(len(knots) - 1):
                    if X_tr[i,j] >= knots[k] and X_tr[i,j]< knots[k + 1]:
                        dtw = k
                transition_matrix[dtw, actual_state, next_state] +=  1
            
            elif not stop_homogenous:
                transition_matrix[j, actual_state, next_state] +=  1

            else:
                transition_matrix[actual_state, next_state] +=  1

    #Normalization
    if not time_homogenous:

        for k in range(0, len(knots) - 1):

            if sum(pi[k,:]) == 0:
                pi[k,:] = [1/3, 1/3, 1/3]
            else:
                pi[k,:] = pi[k,:] / sum(pi[k,:])
            
            if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[k,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[k,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[k,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    elif not stop_homogenous:

        for j in range(T - 1):
            pi = pi/sum(pi)
            
            if transition_matrix[j,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[j,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[j,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[j,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[j,:,:] = (transition_matrix[j,:,:].T/transition_matrix[j,:,:].sum(axis = 1)).T

    else:      
        pi = pi/sum(pi)
        
        #Check first divison by 0
        if transition_matrix.sum(axis = 1)[0] == 0 or transition_matrix.sum(axis = 1)[1] == 0 or transition_matrix.sum(axis = 1)[2] == 0:
            print('error: one or more states not seen in training set. Increase slidewindow_len.')
        else:
            transition_matrix = (transition_matrix.T/transition_matrix.sum(axis = 1)).T

    return pi, transition_matrix

def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogenous = True, stop_homogenous = True):

    future_states = []

    if not time_homogenous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :])
    
    else:
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi)


    future_states.append(first_stop_state)

    for i in range(len(X_tst) - 1):
        current_state = state_dict[future_states[-1]]

        if not time_homogenous:

            for k in range(len(knots) - 1):
                if X_tst[i] >= knots[k] and X_tst[i] < knots[k + 1]:
                    dtw = k
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, current_state, :])
        
        elif not stop_homogenous:
            
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[i, current_state, :])

        else:

            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[current_state, :])

        future_states.append(next_state)

    return future_states

def pred_next_stop_delay(current_state, state_dict, transition_matrix, x_tst, knots, t, time_homogenous = True, stop_homogenous = True):

      if not time_homogenous:

          for k in range(len(knots) - 1):
              if x_tst >= knots[k] and x_tst < knots[k + 1]:
                  dtw = k
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, state_dict[current_state], :])
      
      elif not stop_homogenous:
          
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[t, state_dict[current_state], :])

      else:

          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[state_dict[current_state], :])

In [ ]:
#----- Parameters ----- 
slidewindow_len = 250 #Number of previous days to train on
time_homogenous = True
stop_homogenous = False
if not time_homogenous and not stop_homogenous:
    print('time inhomogenous and stop inhomogenous markov chains not implemented, keep time_homogenous or stop_homogenous True')
    exit() 
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogenous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X_train.shape

for k in range(slidewindow_len, N):

    X_tr = X_train[k-slidewindow_len:k,:]
    Y_tr = y_train[k-slidewindow_len:k,:]
    X_tst = X_train[k,:]
    y_tst = y_train[k,:]
    pi, transition_matrix = parametrization(state_dict, X_tr, Y_tr, knots, time_homogenous, stop_homogenous)


    #Predict the next trip sequence of states
    future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogenous, stop_homogenous)
    
    for t in range(T):

        if y_tst[t] == future_states[t]:
            accu1 += 1
    
    #Predict the state of the next stop
    if not time_homogenous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :]) #Prediction of the first stop state is random
    
    else:
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi) #Prediction of the first stop state is random

    if y_tst[0] == first_state:
        accu2 += 1

    for t in range(0, T - 1):
        next_state = pred_next_stop_delay(y_tst[t], state_dict, transition_matrix, X_tst[t], knots, t, time_homogenous, stop_homogenous)
        
        if y_tst[t] == next_state:
            accu2 += 1


accu1 = accu1 /((N - slidewindow_len) * T)
accu2 = accu2 /((N - slidewindow_len) * T)
print(accu1)
print(accu2)